In [3]:
#PRODUCT CREATION FROM WEB ORDER DEMO

import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Example payload for creating a sale order
payload = {
    "product": {
        "name": f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "width": 500,
        "height": 500, 
        "price": 200.00,
        "reference": f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "components": [
            {
                "name": "Frame A",
                "reference": "TEST_FRAME_A"
            },
            {
                "name": "Glass A", 
                "reference": "TEST_GLASS_A"
            },
            {
                "name": "Passe-Partout A",
                "reference": "TEST_PP_A"
            }
        ]
    }
}

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Create product
product_vals = {
    'name': payload['product']['name'],
    'type': 'consu',  # Changed from 'product' to 'consu' as it's a valid selection value
    'x_studio_width': payload['product']['width'],
    'x_studio_height': payload['product']['height'],
    'list_price': payload['product']['price'],
    'default_code': payload['product']['reference'],
    'categ_id': 8,  # Set category ID to 8
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Get the product template ID from the created product
product_data = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'read', [product_id], {'fields': ['product_tmpl_id']}
)
product_tmpl_id = product_data[0]['product_tmpl_id'][0]

# Create components and BOM
bom_components = []
bom_operations = []
width = payload['product']['width']
height = payload['product']['height']
surface = width * height
circumference = 2 * (width + height)

print(f"Surface: {surface} mm²")
print(f"Surface: {surface/1000000} m²")
print(f"Circumference: {circumference} mm")

# Convert dimensions to meters for duration rules
surface_m2 = surface / 1000000  # Convert mm² to m²
circumference_m = circumference / 1000  # Convert mm to m

for component in payload['product']['components']:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    if not component_ids:
        # Create component if it doesn't exist
        component_vals = {
            'name': component['name'],
            'type': 'product',
            'default_code': component['reference'],
            'categ_id': 8,
        }
        component_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'create', [component_vals])
    else:
        component_id = component_ids[0]
    
    # Get component details to check price computation method and associated service
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation', 'x_studio_associated_service', 'x_studio_associated_service_duration_rule']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = circumference_m
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = surface_m2
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

    # Handle associated service/operation
    if component_info[0]['x_studio_associated_service']:
        service_id = component_info[0]['x_studio_associated_service'][0]
        
        # Get service details
        service_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services', 'read',
            [service_id],
            {'fields': ['x_name', 'x_soort', 'x_studio_associated_work_center']})

        # Get duration rules directly from component
        duration_rules = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services_duration_rules', 'read',
            [component_info[0]['x_studio_associated_service_duration_rule']],
            {'fields': ['x_omtrek', 'x_oppervlakte', 'x_duurtijd_totaal']})

        # Find appropriate duration based on x_soort
        if service_info[0]['x_soort'] == 'Oppervlakte':
            relevant_value = surface_m2  # Use m² value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_oppervlakte'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_oppervlakte'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']
        else:  # 'Omtrek'
            relevant_value = circumference_m  # Use m value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_omtrek'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_omtrek'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']

        # Convert duration from seconds to minutes and calculate MM:SS format
        duration_minutes = duration_seconds / 60
        minutes = int(duration_minutes)
        seconds = int((duration_minutes - minutes) * 60)
        odoo_display = f"{minutes:02d}:{seconds:02d}"
        print(f"Converting duration for {service_info[0]['x_name']}: {duration_seconds} seconds = {duration_minutes} minutes (will display as {odoo_display} in Odoo)")

        # Add operation to BOM
        bom_operations.append((0, 0, {
            'name': service_info[0]['x_name'],
            'time_cycle_manual': duration_minutes,
            'workcenter_id': service_info[0]['x_studio_associated_work_center'][0] if service_info[0]['x_studio_associated_work_center'] else False
        }))

# Create Bill of Materials using the template ID
bom_vals = {
    'product_tmpl_id': product_tmpl_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components,
    'operation_ids': bom_operations,
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Create sale order
order_vals = {
    'partner_id': 1,  # Default customer ID, adjust as needed
    'order_line': [(0, 0, {
        'product_id': product_id,
        'product_uom_qty': 1,
        'price_unit': payload['product']['price']
    })]
}

order_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'create', [order_vals])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Created sale order ID: {order_id}")
print("Sale order from web simulation finished")

Surface: 250000 mm²
Surface: 0.25 m²
Circumference: 2000 mm
Converting duration for Cutting Wood: 300 seconds = 5.0 minutes (will display as 05:00 in Odoo)
Converting duration for Cutting Glass: 620 seconds = 10.333333333333334 minutes (will display as 10:20 in Odoo)
Created product ID: 14810
Created BOM ID: 57
Created sale order ID: 50
Sale order from web simulation finished


In [5]:
#PRODUCT CREATION FROM EXISTING SALE ORDER (MANUAL PROCESS)

import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Get sale order
sale_order_id = 48
sale_order = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'read',
    [sale_order_id],
    {'fields': ['order_line']})

# Get first product from sale order
order_line = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order.line', 'read',
    [sale_order[0]['order_line'][0]],
    {'fields': ['product_id', 'price_unit']})

product_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'read',
    [order_line[0]['product_id'][0]],
    {'fields': ['x_studio_width', 'x_studio_height', 'name', 'default_code']})

# Get product details
width = product_info[0]['x_studio_width']
height = product_info[0]['x_studio_height']
price = order_line[0]['price_unit']

# Get BOM for the product
bom_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'search',
    [[['product_tmpl_id', '=', product_info[0]['id']]]])

bom_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'read',
    [bom_ids[0]],
    {'fields': ['bom_line_ids']})

# Get components from BOM
components = []
for bom_line_id in bom_info[0]['bom_line_ids']:
    bom_line = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'mrp.bom.line', 'read',
        [bom_line_id],
        {'fields': ['product_id']})
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [bom_line[0]['product_id'][0]],
        {'fields': ['name', 'default_code']})
    components.append({
        'name': component_info[0]['name'],
        'reference': component_info[0]['default_code']
    })

# Create new product with copied details
product_vals = {
    'name': f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
    'type': 'consu',
    'x_studio_width': width,
    'x_studio_height': height,
    'list_price': price,
    'default_code': f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    'categ_id': 8,
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Create components and BOM
bom_components = []
bom_operations = []
surface = width * height
circumference = 2 * (width + height)

print(f"Surface: {surface} mm²")
print(f"Surface: {surface/1000000} m²")
print(f"Circumference: {circumference} mm")

# Convert dimensions to meters for duration rules
surface_m2 = surface / 1000000  # Convert mm² to m²
circumference_m = circumference / 1000  # Convert mm to m

for component in components:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    component_id = component_ids[0]
    
    # Get component details to check price computation method and associated service
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation', 'x_studio_associated_service', 'x_studio_associated_service_duration_rule']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = 2 * (width + height)
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = width * height
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

    # Handle associated service/operation
    if component_info[0]['x_studio_associated_service']:
        service_id = component_info[0]['x_studio_associated_service'][0]
        
        # Get service details
        service_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services', 'read',
            [service_id],
            {'fields': ['x_name', 'x_soort', 'x_studio_associated_work_center']})

        # Get duration rules directly from component
        duration_rules = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'x_services_duration_rules', 'read',
            [component_info[0]['x_studio_associated_service_duration_rule']],
            {'fields': ['x_omtrek', 'x_oppervlakte', 'x_duurtijd_totaal']})

        # Find appropriate duration based on x_soort
        if service_info[0]['x_soort'] == 'Oppervlakte':
            relevant_value = surface_m2  # Use m² value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_oppervlakte'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_oppervlakte'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']
        else:  # 'Omtrek'
            relevant_value = circumference_m  # Use m value
            rules_sorted = sorted(duration_rules, key=lambda x: x['x_omtrek'])
            matching_rule = next((rule for rule in rules_sorted if rule['x_omtrek'] >= relevant_value), rules_sorted[-1])
            duration_seconds = matching_rule['x_duurtijd_totaal']

        # Convert duration from seconds to minutes and calculate MM:SS format
        duration_minutes = duration_seconds / 60
        minutes = int(duration_minutes)
        seconds = int((duration_minutes - minutes) * 60)
        odoo_display = f"{minutes:02d}:{seconds:02d}"
        print(f"Converting duration for {service_info[0]['x_name']}: {duration_seconds} seconds = {duration_minutes} minutes (will display as {odoo_display} in Odoo)")

        # Add operation to BOM
        bom_operations.append((0, 0, {
            'name': service_info[0]['x_name'],
            'time_cycle_manual': duration_minutes,
            'workcenter_id': service_info[0]['x_studio_associated_work_center'][0] if service_info[0]['x_studio_associated_work_center'] else False
        }))

# Create Bill of Materials
bom_vals = {
    'product_tmpl_id': product_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components,
    'operation_ids': bom_operations
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Update existing sale order with new product
models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'write',
    [sale_order_id, {
        'order_line': [(1, sale_order[0]['order_line'][0], {
            'product_id': product_id,
            'product_uom_qty': 1,
            'price_unit': price
        })]
    }])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Updated sale order ID: {sale_order_id}")
print("Manual process simulation finished")

Surface: 10000.0 mm²
Surface: 0.01 m²
Circumference: 400.0 mm
Converting duration for Cutting Wood: 300 seconds = 5.0 minutes (will display as 05:00 in Odoo)
Converting duration for Cutting Glass: 620 seconds = 10.333333333333334 minutes (will display as 10:20 in Odoo)
Created product ID: 73
Created BOM ID: 53
Updated sale order ID: 48
Manual process simulation finished


In [ ]:
#PRODUCT CREATION DEMO (OLD, before operations)


import os
from dotenv import load_dotenv
import xmlrpc.client
from datetime import datetime

# Example payload for creating a sale order
payload = {
    "product": {
        "name": f"Finished Product {datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "width": 50,
        "height": 100, 
        "price": 200.00,
        "reference": f"TEST_FINISHED_PRODUCT_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "components": [
            {
                "name": "Frame A",
                "reference": "TEST_FRAME_B"
            },
            {
                "name": "Glass A", 
                "reference": "TEST_GLASS_A"
            },
            {
                "name": "Passe-Partout A",
                "reference": "TEST_PP_A"
            }
        ]
    }
}


# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Create product
product_vals = {
    'name': payload['product']['name'],
    'type': 'consu',  # Changed from 'product' to 'consu' as it's a valid selection value
    'x_studio_width': payload['product']['width'],
    'x_studio_height': payload['product']['height'],
    'list_price': payload['product']['price'],
    'default_code': payload['product']['reference'],
    'categ_id': 8,  # Set category ID to 8
}

product_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.product', 'create', [product_vals])

# Create components and BOM
bom_components = []
width = payload['product']['width']
height = payload['product']['height']

for component in payload['product']['components']:
    # Search for existing component
    component_ids = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'search',
        [[['default_code', '=', component['reference']]]])
    
    if not component_ids:
        # Create component if it doesn't exist
        component_vals = {
            'name': component['name'],
            'type': 'product',
            'default_code': component['reference'],
            'categ_id': 8,
        }
        component_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'create', [component_vals])
    else:
        component_id = component_ids[0]
    
    # Get component details to check price computation method
    component_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.product', 'read',
        [component_id],
        {'fields': ['x_studio_price_computation']})
    
    # Calculate quantity based on price computation method
    if component_info and component_info[0]['x_studio_price_computation'] == 'Circumference':
        quantity = 2 * (width + height)
    elif component_info and component_info[0]['x_studio_price_computation'] == 'Surface':
        quantity = width * height
    else:
        quantity = 1
    
    # Add to BOM components
    bom_components.append((0, 0, {
        'product_id': component_id,
        'product_qty': quantity,
    }))

# Create Bill of Materials
bom_vals = {
    'product_tmpl_id': product_id,
    'product_qty': 1,
    'type': 'normal',
    'bom_line_ids': bom_components
}

bom_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'mrp.bom', 'create', [bom_vals])

# Create sale order
order_vals = {
    'partner_id': 1,  # Default customer ID, adjust as needed
    'order_line': [(0, 0, {
        'product_id': product_id,
        'product_uom_qty': 1,
        'price_unit': payload['product']['price']
    })]
}

order_id = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'sale.order', 'create', [order_vals])

print(f"Created product ID: {product_id}")
print(f"Created BOM ID: {bom_id}")
print(f"Created sale order ID: {order_id}")

In [ ]:
#FETCHING SUPPLIER INFO TO MANAGE SEQUENCE

import os
from dotenv import load_dotenv
import xmlrpc.client

# Reload environment variables
load_dotenv(override=True)

## Odoo connection details from environment variables
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB') 
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

# Create object proxy
models = xmlrpc.client.ServerProxy(f'{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object')

# Fetch product with ID 4 and get seller_ids field
product = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
    'product.template', 'read',
    [4],
    {'fields': ['seller_ids']}
)

if product:
    seller_ids = product[0]['seller_ids']
    print("Seller IDs:", seller_ids)
    
    # Fetch detailed supplier info for each seller_id
    supplier_info = models.execute_kw(JUSTFRAMEIT_ODOO_DB, uid, JUSTFRAMEIT_ODOO_API_KEY,
        'product.supplierinfo', 'read',
        [seller_ids],
        {'fields': ['id', 'display_name', 'sequence', 'product_tmpl_id']}
    )
    print("\nDetailed Supplier Information:")
    for info in supplier_info:
        print(info)
else:
    print("Product not found")

Seller IDs: [1, 4, 2]

Detailed Supplier Information:
{'id': 1, 'display_name': 'Louis Test (0.0 Units - 15.00\xa0€)', 'sequence': 1, 'product_tmpl_id': [4, 'Frame A']}
{'id': 4, 'display_name': 'Louis Test 3 (0.0 Units - 19.00\xa0€)', 'sequence': 2, 'product_tmpl_id': [4, 'Frame A']}
{'id': 2, 'display_name': 'Louis Test 2 (0.0 Units - 6.00\xa0€)', 'sequence': 3, 'product_tmpl_id': [4, 'Frame A']}


In [11]:
#EXPORTING SPREADSHEET FINAL APPROACH
#(fetching spreadsheet.dashboard not working as it only get the odoo. formula, without the associated value)
#Using spreadsheet.dashboard.share is a workaround

import os
import json
import base64
import xmlrpc.client
from dotenv import load_dotenv
import xlsxwriter
import openpyxl

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object")

# Step 1: Get spreadsheet JSON (binary field)
spreadsheets = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'spreadsheet.dashboard.share',
    'search_read',
    [[('display_name', '=', 'Price List 1')]],
    {'fields': ['id', 'display_name', 'spreadsheet_data'], 'limit': 1, 'order': 'create_date desc'}
)

if not spreadsheets:
    raise Exception("❌ No spreadsheet found with name 'Price List 1'.")

spreadsheet = spreadsheets[0]
print(f"✅ Found spreadsheet: {spreadsheet['display_name']} (ID {spreadsheet['id']})")

# Parse the spreadsheet data
data = json.loads(spreadsheet['spreadsheet_data'])

# Create workbook
workbook = xlsxwriter.Workbook('pricelist3.xlsx')

for sheet in data['sheets']:
    worksheet = workbook.add_worksheet(sheet['name'])
    worksheet.use_future_functions = True
    
    # Set values
    for cell_ref, value in sheet['cells'].items():
        col = openpyxl.utils.column_index_from_string(''.join(filter(str.isalpha, cell_ref))) - 1
        row = int(''.join(filter(str.isdigit, cell_ref))) - 1
        
        # Convert string numbers to float if possible
        if isinstance(value, str):
            try:
                value = float(value)
            except ValueError:
                pass
                
        worksheet.write(row, col, value)

workbook.close()
print(f"✅ Spreadsheet exported successfully to pricelist3.xlsx")


✅ Found spreadsheet: Price List 1 (ID 35)
✅ Spreadsheet exported successfully to pricelist3.xlsx


In [ ]:
#DELETING SHARED SPREADSHEETS

import os
import json
import base64
import xmlrpc.client
from dotenv import load_dotenv
from openpyxl import Workbook

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object")

# Step 1: Search for all Price List 1 spreadsheets
spreadsheets = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'spreadsheet.dashboard.share',
    'search_read',
    [[('display_name', '=', 'Price List 1')]],
    {'fields': ['id']}
)

if not spreadsheets:
    print("No spreadsheets found with name 'Price List 1'")
else:
    # Get all IDs
    spreadsheet_ids = [s['id'] for s in spreadsheets]
    print(f"Found {len(spreadsheet_ids)} spreadsheets to delete")
    
    # Delete all found spreadsheets
    try:
        models.execute_kw(
            JUSTFRAMEIT_ODOO_DB,
            uid,
            JUSTFRAMEIT_ODOO_API_KEY,
            'spreadsheet.dashboard.share',
            'unlink',
            [spreadsheet_ids]
        )
        print(f"✅ Successfully deleted {len(spreadsheet_ids)} spreadsheets")
    except Exception as e:
        print(f"❌ Failed to delete spreadsheets: {str(e)}")


Found 14 spreadsheets to delete
✅ Successfully deleted 14 spreadsheets


In [ ]:
#COMPUTE PRICE FROM BOM

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Get initial cost before computing BOM
product_id = 60
initial_cost = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'product.template',
    'read',
    [[product_id]],
    {'fields': ['standard_price']}
)[0]['standard_price']

print(f"Initial cost: {initial_cost}")

# Compute BOM cost for product template ID 60
try:
    # Ignore return value from button_bom_cost
    models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'button_bom_cost',
        [[product_id]]
    )
    
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")
        
except:
    # Silently continue if button_bom_cost returns None
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")

Initial cost: 1000.0
New cost: 1334417.22
✅ Cost change detected - BOM computation successful


In [ ]:
#COMPUTE PRICE FROM BOM

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

# Connect to Odoo with allow_none=True to handle None values
common = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
uid = common.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})

if not uid:
    raise Exception("❌ Failed to authenticate to Odoo. Please check credentials.")

models = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)

# Get initial cost before computing BOM
product_id = 60
initial_cost = models.execute_kw(
    JUSTFRAMEIT_ODOO_DB,
    uid,
    JUSTFRAMEIT_ODOO_API_KEY,
    'product.template',
    'read',
    [[product_id]],
    {'fields': ['standard_price']}
)[0]['standard_price']

print(f"Initial cost: {initial_cost}")

# Compute BOM cost for product template ID 60
try:
    # Ignore return value from button_bom_cost
    models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'button_bom_cost',
        [[product_id]]
    )
    
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")
        
except:
    # Silently continue if button_bom_cost returns None
    # Get new cost after computation
    new_cost = models.execute_kw(
        JUSTFRAMEIT_ODOO_DB,
        uid,
        JUSTFRAMEIT_ODOO_API_KEY,
        'product.template',
        'read',
        [[product_id]],
        {'fields': ['standard_price']}
    )[0]['standard_price']
    
    print(f"New cost: {new_cost}")
    
    if new_cost != initial_cost:
        print("✅ Cost change detected - BOM computation successful")
    else:
        print("⚠️ Warning: No cost change detected after BOM computation")

In [ ]:
import os
from dotenv import load_dotenv
import xmlrpc.client

# Load environment variables
load_dotenv()

# Odoo Configuration
DECILO_ODOO_URL = os.getenv('DECILO_ODOO_URL')
DECILO_ODOO_DB = os.getenv('DECILO_ODOO_DB')
DECILO_ODOO_USERNAME = os.getenv('DECILO_ODOO_USERNAME')
DECILO_ODOO_API_KEY = os.getenv('DECILO_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(DECILO_ODOO_DB, DECILO_ODOO_USERNAME, DECILO_ODOO_API_KEY, {})

# Create object proxy
models = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/object')

# Search for product variants
product_id = 13  # Replace with actual product ID
product = models.execute_kw(DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
    'product.template', 'read',
    [product_id],
    {'fields': ['name', 'attribute_line_ids']}
)

if product:
    # Get attribute lines details
    attr_lines = models.execute_kw(DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
        'product.template.attribute.line', 'read',
        [product[0]['attribute_line_ids']],
        {'fields': ['attribute_id', 'value_ids']}
    )
    
    print("\nProduct Variants:")
    for line in attr_lines:
        # Get attribute values
        values = models.execute_kw(DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
            'product.attribute.value', 'read',
            [line['value_ids']],
            {'fields': ['name']}
        )
        print(f"Attribute: {line['attribute_id'][1]}")
        print(f"Values: {[val['name'] for val in values]}\n")
else:
    print("Product not found")


Product Variants:
Attribute: Sides
Values: ['Stereo', 'Mono R', 'Mono L']

Attribute: Venting
Values: ['No venting', '0.8', '1.0', '1.2', '1.5', '1.8', '2.0', '2.2', '2.5', '2.8', '3.0', '3.5']

Attribute: Material
Values: ['Acrylic', 'Soft']

Attribute: Lock
Values: ['Yes', 'No']

Attribute: Ear Impression Type
Values: ['Impression Paste - Scan 3D', 'Impression Paste - Sending ear impressions', 'Ear impressions with previous order', 'Otoscan']



In [22]:
import os
from dotenv import load_dotenv
import xmlrpc.client

# Load environment variables
load_dotenv()

# Odoo Configuration
DECILO_ODOO_URL = os.getenv('DECILO_ODOO_URL')
DECILO_ODOO_DB = os.getenv('DECILO_ODOO_DB')
DECILO_ODOO_USERNAME = os.getenv('DECILO_ODOO_USERNAME')
DECILO_ODOO_API_KEY = os.getenv('DECILO_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(DECILO_ODOO_DB, DECILO_ODOO_USERNAME, DECILO_ODOO_API_KEY, {})

# Create object proxy
models = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/object')


models.execute_kw(
    DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
    'sale.order', 'message_post',
    [[8]],  # Order ID
    {
        'body': "<p>test<br/>test<br/>test</p>",
        'body_is_html': True,             # 🔑 ensures it's rendered as HTML
        'message_type': 'comment',
        'subtype_xmlid': 'mail.mt_comment',
    }
)

models.execute_kw(
    DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
    'sale.order', 'message_post',
    [[8]],  # Order ID
    {
        'body': "<p>test<br/>test<br/>test</p>",
        'body_is_html': True,                # keep HTML rendering
        'message_type': 'comment',
        'subtype_xmlid': 'mail.mt_note',     # 🔑 internal note
    }
)




[4161]

In [13]:
import os
from dotenv import load_dotenv
import xmlrpc.client

# Load environment variables
load_dotenv()

# Odoo Configuration
DECILO_ODOO_URL = os.getenv('DECILO_ODOO_URL')
DECILO_ODOO_DB = os.getenv('DECILO_ODOO_DB')
DECILO_ODOO_USERNAME = os.getenv('DECILO_ODOO_USERNAME')
DECILO_ODOO_API_KEY = os.getenv('DECILO_ODOO_API_KEY')

# Connect to Odoo
common = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/common')
models = xmlrpc.client.ServerProxy(f'{DECILO_ODOO_URL}/xmlrpc/2/object')

# Authenticate
uid = common.authenticate(DECILO_ODOO_DB, DECILO_ODOO_USERNAME, DECILO_ODOO_API_KEY, {})

# Fetch messages for order 8
messages = models.execute_kw(
    DECILO_ODOO_DB, uid, DECILO_ODOO_API_KEY,
    'mail.message', 'search_read',
    [[('model', '=', 'sale.order'), ('res_id', '=', 8)]],
    {'fields': ['body', 'date', 'author_id']}
)

print("Messages for order #8:")
for msg in messages:
    print(f"\nDate: {msg['date']}")
    print(f"Author: {msg['author_id'][1] if msg['author_id'] else 'System'}")
    print(f"Message:\n{msg['body']}")

Messages for order #8:

Date: 2025-09-16 13:58:48
Author: Louis Dresse
Message:
<p>&lt;p&gt;test&lt;br&gt;test&lt;br&gt;test&lt;/p&gt;</p>

Date: 2025-09-16 13:58:15
Author: Louis Dresse
Message:
<p>test<br>test<br>test</p>

Date: 2025-09-16 13:57:31
Author: Louis Dresse
Message:
<p>&lt;div style='font-family: Arial, sans-serif;'&gt;&lt;h3 style='color: #2c3e50; margin-bottom: 15px;'&gt;📋 New Portal Order Details&lt;/h3&gt;&lt;div style='margin-bottom: 20px;'&gt;&lt;p style='font-weight: bold; color: #34495e;'&gt;Product: BTE - Full Shell&lt;/p&gt;&lt;div style='background-color: #f8f9fa; padding: 10px; border-radius: 5px; margin-bottom: 15px;'&gt;&lt;p style='margin: 0;'&gt;&lt;b&gt;👤 Patient Information&lt;/b&gt;&lt;/p&gt;&lt;p style='margin: 5px 0;'&gt;Name: 123a123 123a123&lt;/p&gt;&lt;p style='margin: 5px 0;'&gt;Audiolog: a123a123&lt;/p&gt;&lt;p style='margin: 5px 0;'&gt;Auditive Center: 1z23s1df&lt;/p&gt;&lt;/div&gt;&lt;div style='background-color: #f8f9fa; padding: 10px; border-